In [1]:
import os, time
import gymnasium as gym
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecMonitor
from stable_baselines3 import PPO

from callback import SaveOnBestTrainingRewardCallback

def train(env_id, log_base_dir="logs", model_base_dir="models", model_name=None, total_timesteps=100000):
    """
    Train a PPO agent on the specified environment.
    
    Args:
        env_id (str): Environment ID to train on
        log_base_dir (str): Base directory for logs
        model_base_dir (str): Base directory for saving models
        model_name (str): Name for the model file
        total_timesteps (int): Total timesteps for training
    """   
    # Create log directory and model directory
    script_dir = os.path.dirname(os.path.abspath(__file__))
    log_dir = os.path.join(script_dir, log_base_dir, env_id)
    model_dir = os.path.join(script_dir, model_base_dir)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)
    
    # Environment
    n_envs = 64
    env = make_vec_env(env_id, n_envs=n_envs)
    env = VecMonitor(env, log_dir)

    # Agent Model
    if model_name is None:    
        model_name = env_id + "_PPO"
    
    policy_kwargs = {
        'log_std_init':-2,
        'ortho_init': False,
    }
    
    model = PPO(
        policy="MlpPolicy",
        env=env,
        # -------------------------------------------------
        learning_rate=1e-3,
        n_steps = 500000,
        gamma=0.9,
        gae_lambda=0.95,
        batch_size=64,
        n_epochs=10,
        ent_coef=0.0,
        clip_range=0.2,
        clip_range_vf=None,
        normalize_advantage=True,
        vf_coef=0.5,
        max_grad_norm=0.5,
        use_sde=True,
        sde_sample_freq=4,
        rollout_buffer_class=None,
        rollout_buffer_kwargs=None,
        target_kl=None,
        stats_window_size=100,
        policy_kwargs=policy_kwargs,
        # -------------------------------------------------
        tensorboard_log = log_dir,
        verbose=1,
        seed=None,
        device='auto',
    )            


    # Train
    callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)   
    model.learn(
        total_timesteps=total_timesteps,
        callback=callback,
        log_interval=4,
        tb_log_name="PPO",
        reset_num_timesteps=True,
        progress_bar=True,
    )

    # Save the trained model
    save_path = os.path.join(model_dir, model_name)
    model.save(save_path)

    # close the environment
    env.close()


def run(env_id, model_base_dir="models", model_name=None, n_episodes=5):
    """
    Run a trained PPO agent on the specified environment.
    
    Args:
        env_id (str): Environment ID to run on
        model_base_dir (str): Base directory for loading models
        model_name (str): Name of the model file
        n_episodes (int): Number of episodes to run
    """
    # Environment
    env = gym.make(env_id,
                   xml_file='humanoid.xml',
                   forward_reward_weight=1.0,
                   ctrl_cost_weight=0.1,
                   contact_cost_weight=5e-7,
                   healthy_reward=5.0,
                   terminate_when_unhealthy=True,
                   healthy_z_range=(1.0, 2.0),
                   reset_noise_scale=1e-2,
                   exclude_current_positions_from_observation=True,
                   )
    
    # Model
    script_dir = os.path.dirname(os.path.abspath(__file__))
    if model_name is None:
        model_name = env_id + "_PPO"
    model_path = os.path.join(script_dir, model_base_dir, model_name)
    model = PPO.load(model_path, env)

    # Run    
    for episode in range(n_episodes):
        obs, info = env.reset()

        episode_reward = 0
        n_standing_steps = 0
        while True:
            action, state = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            time.sleep(0.01)
            episode_reward += reward
            
            # Count steps where pendulum is standing (reward > threshold)
            if reward > -1.0:
                n_standing_steps += 1
                
            if terminated or truncated:
                time.sleep(1.0)
                print(f"Episode {episode + 1}: Total reward = {episode_reward:.2f}, Standing steps = {n_standing_steps}/200")
                episode_reward = 0
                n_standing_steps = 0
                break
            
    # close the environment
    env.close()


# if __name__ == "__main__":
#     env_id = "Humanoid-v5"
    
#     train(env_id)
#     # run(env_id)

In [5]:
import os

log_base_dir="logs"
env_id = "Humanoid-v5"
model_base_dir="models"
model_name=None
total_timesteps=100000

# script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = os.getcwd()
log_dir = os.path.join(script_dir, log_base_dir, env_id)
model_dir = os.path.join(script_dir, model_base_dir)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# Environment
n_envs = 64
env = make_vec_env(env_id, n_envs=n_envs)
env = VecMonitor(env, log_dir)

# Agent Model
if model_name is None:    
    model_name = env_id + "_PPO"

policy_kwargs = {
    'log_std_init':-2,
    'ortho_init': False,
}

model = PPO(
    policy="MlpPolicy",
    env=env,
    # -------------------------------------------------
    learning_rate=1e-3,
    n_steps = 500000,
    gamma=0.9,
    gae_lambda=0.95,
    batch_size=64,
    n_epochs=10,
    ent_coef=0.0,
    clip_range=0.2,
    clip_range_vf=None,
    normalize_advantage=True,
    vf_coef=0.5,
    max_grad_norm=0.5,
    use_sde=True,
    sde_sample_freq=4,
    rollout_buffer_class=None,
    rollout_buffer_kwargs=None,
    target_kl=None,
    stats_window_size=100,
    policy_kwargs=policy_kwargs,
    # -------------------------------------------------
    tensorboard_log = log_dir,
    verbose=1,
    seed=None,
    device='auto',
)            


# Train
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)   
# model.learn(
#     total_timesteps=total_timesteps,
#     callback=callback,
#     log_interval=4,
#     tb_log_name="PPO",
#     reset_num_timesteps=True,
#     progress_bar=True,
# )

# Save the trained model
save_path = os.path.join(model_dir, model_name)
model.save(save_path)

# close the environment
env.close()

/home/kimm-linux/Documents/Cource/2025_PHD/AdvancedRL/termPro/.venv/lib/python3.11/site-packages/stable_baselines3/
common/vec_env/vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you 
are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(

Using cuda device

/home/kimm-linux/Documents/Cource/2025_PHD/AdvancedRL/termPro/.venv/lib/python3.11/site-packages/stable_baselines3/
common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended 
to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See 
https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export 
CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and 
the training might take longer than on CPU.
  warnings.warn(